### 트리 13306

In [17]:
import sys
# input = lambda: sys.stdin.readline().rstrip()

n, q = map(int, input().split())
d = {}
for i in range(n - 1):
    d.update({i + 2: int(input())})
queries = [list(map(int, input().split())) for _ in range(q + n - 1)]

parent = list(range(n + 1))
rank = [0] * (n + 1)

def find(v):
    if v == parent[v]:
        return v
    parent[v] = find(parent[v])
    return parent[v]

def union(a, b):
    if rank[a] >= rank[b]:
        if rank[a] == rank[b]:
            rank[a] += 1
        parent[b] = a
    else:
        parent[a] = b
        
result = []
for i in range(q + n - 2, -1, -1):
    query = queries[i]
    if query[0] == 1:
        if find(query[1]) == find(query[2]):
            result.append('YES')
        else:
            result.append('NO')
    else:
        union(find(query[1]), find(d[query[1]]))
        
print('\n'.join(reversed(result)))

NO
YES
YES
NO
YES


### 꼬리달린 박성원숭이 2105

In [ ]:
먼저 연결되어있는 경우를 다 받아서 dict으로 카운트까지 관리함 (정렬시켜서 저장)
또한 동시에 어느 손으로 어느 원숭이를 잡고 있는지에 대한 정보도 저장함
제거되는 경우 다 받아서 원숭이로 다시 변환하고 저장함 (순서 기억함)
맨 위의 dict 복사 후 dict에서 빼줌
dict 다 돌면서 값이 1 이상이면 전부 연결해줌

find, union 만듦
정답을 담을 리스트 만듦 (처음에는 전부 -1)
parent 만듦
temp_connected [{}, {}, {} ...] 만듦
지우는 쿼리 마지막부터 연결해주기 시작함

1은 temp_connected에서 집합 처리 X
1 만나면 만난 것 + 가지고 있던 집합을 temp_connected에서 집합 처리 해줌
2 안만나면 번호 작은 노드에 집합 다 옮기고 큰 번호 추가하고 temp_connected에서
  큰 번호 노드 집합 set()으로 만들어줌

In [ ]:
import sys
input = lambda: sys.stdin.readline().rstrip()

n, m = map(int, input().split())
st = set()
left_right_info = dict()

for i in range(n):
    left, right = map(int, input().split())
    if left != -1:
        st.add((i + 1, left))
        left_right_info[(i + 1, 1)] = left
    if right != -1:
        st.add((i + 1, right))
        left_right_info[(i + 1, 2)] = right  
        
deleted = []
for _ in range(m):
    a, hand = list(map(int, input().split()))
    b = left_right_info[(a, hand)]
    deleted.append((a, b))
    st.discard((a, b))
del left_right_info

def find(v):
    if parent[v] == v:
        return v
    parent[v] = find(parent[v])
    return parent[v]

def union(ra, rb, option=0):
    if rb < ra:
        ra, rb = rb, ra
    
    parent[rb] = ra
    
    if ra != 1:
        for v in temp_connected[rb]:
            temp_connected[ra].add(v)
        temp_connected[ra].add(rb)
    else:
        if option:
            temp_connected[rb].add(rb)
            for node in temp_connected[rb]:
                result[node - 1] = t
    temp_connected[rb] = set()
    
parent = list(range(n + 1))
temp_connected = [set() for _ in range(n + 1)]
result = [-1] * n

for a, b in st:
    if a > b:
        a, b = b, a
    ra, rb = find(a), find(b)
    if ra == rb:
        continue
    union(ra, rb)
    
for t in range(m - 1, -1, -1):
    a, b = deleted[t]
    if a > b:
        a, b = b, a
    ra, rb = find(a), find(b)
    if ra == rb:
        continue
    union(ra, rb, 1)
    
print('\n'.join(map(str, result)))

In [ ]:
3 2
-1 3
3 -1
1 2
1 2
3 1
-1
1
1

In [39]:
left_right_info

{(1, 2): 3, (2, 1): 3, (3, 1): 1, (3, 2): 2}

In [78]:
a = set([3, 4])
b = set([2, 5])



In [64]:
a.union(b)

{2, 3, 4, 5}

In [65]:
a

{3, 4}

In [75]:
a.difference_update(b)

AttributeError: 'NoneType' object has no attribute 'difference_update'

In [87]:
result = [[-1] for _ in range(10)]

In [81]:
result

[[-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1], [-1]]

In [91]:
id(result[3])

789939372168

In [89]:
result[3] = result[4]
result[5] = result[6]
result[3] = result[5]

[[-1], [-1], [-1], [2], [2], [2], [2], [-1], [-1], [-1]]

In [84]:
result

[[-1], [-1], [-1], [2], [2], [2], [-1], [2], [-1], [-1]]

In [77]:
b

{2, 5}

### 마블 2843

In [ ]:
먼저 모든 간선을 딕셔너리로 d[출발] = 도착 방식으로 저장함
d의 복사본 new_d를 만듦
쿼리 돌리면서 리스트로 저장하며, 2가 나올 경우 해당 간선을 위의 new_d에서 제거함

new_d에서에 남아있는 간선들의 key와 value가 각각 a, b라면 a의 parent는 b인 식으로 연결해줌.
이때 b의 root가 a인 경우 a의 parent를 바로 0으로 만들어줌 (사이클임을 표시하기 위해)

쿼리 종류가 1인 경우 해당 쿼리의 루트를 result로 저장함 (0인 경우 답은 CIKLUS)
쿼리 종류가 2인 경우 d를 통해 넘어가는 간선을 찾아서 위와 같은 방식으로 연결해줌

In [12]:
import sys
sys.setrecursionlimit(300001)
# input = lambda: sys.stdin.readline().rstrip()

n = int(input())
outs = list(map(int, input().split()))
d = dict()
for i in range(n):
    if outs[i] != 0:
        d[i + 1] = outs[i]
del outs
q = int(input())

queries = []
deleted = dict()
for _ in range(q):
    qt, num = map(int, input().split())
    queries.append((qt, num))
    if qt == 2:
        deleted[num] = d[num]
        del d[num] 
        
def find(v):
    if p[v] == v:
        return v
    p[v] = find(p[v])
    return p[v]

def union(a, rb):
    if a == rb:
        p[a] = 0
    else:
        p[a] = rb
    
p = list(range(n + 1))

d_len = len(d)
for _ in range(d_len):
    a, b = d.popitem()
    rb = find(b)
    union(a, rb)
del d

result = []
for _ in range(q):
    qt, a = queries.pop()
    if qt == 1:
        ra = find(a)
        result.append(ra)
    else:
        b = deleted[a]
        del deleted[a]
        rb = find(b)
        union(a, rb)
del p
del queries

for v in reversed(result):
    if v:
        print(v)
    else:
        print('CIKLUS')

3
2 3 1
7
1 1
1 2
2 1
1 2
1 1
2 2
1 2
CIKLUS
CIKLUS
1
1
2
